In [1]:
from ib_insync import * 
import pandas as pd
import numpy as np
import datetime as dt
import os
from IPython.display import display, clear_output
import pytz
util.startLoop()

In [2]:
ib = IB()
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [3]:
# strategy parameters
sma_s = 2
sma_l = 5
freq = "1 min"
units = 1000
end_time = dt.time(21, 59, 0) # stop condition -> insert your time!!!
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [4]:
ib.qualifyContracts(contract)

[Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')]

In [5]:
dt.datetime.now()

datetime.datetime(2023, 11, 6, 17, 32, 58, 946430)

In [6]:
tz = pytz.timezone('Asia/Shanghai')

In [7]:
session_start = pd.to_datetime(dt.datetime.now(tz))
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting=freq,
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
last_bar = bars[-1].date

In [8]:
df = pd.DataFrame(bars)

In [9]:
df

,date,open,high,low,close,volume,average,barCount
0,2023-11-05 22:15:00+00:00,1.072600,1.072720,1.072385,1.072475,-1.0,-1.0,-1
1,2023-11-05 22:16:00+00:00,1.072480,1.072545,1.072475,1.072490,-1.0,-1.0,-1
2,2023-11-05 22:17:00+00:00,1.072490,1.072545,1.072390,1.072465,-1.0,-1.0,-1
3,2023-11-05 22:18:00+00:00,1.072465,1.072470,1.072465,1.072470,-1.0,-1.0,-1
4,2023-11-05 22:19:00+00:00,1.072470,1.072475,1.072470,1.072475,-1.0,-1.0,-1
...,...,...,...,...,...,...,...,...
673,2023-11-06 09:28:00+00:00,1.074790,1.074790,1.074575,1.074605,-1.0,-1.0,-1
674,2023-11-06 09:29:00+00:00,1.074605,1.074650,1.074560,1.074625,-1.0,-1.0,-1
675,2023-11-06 09:30:00+00:00,1.074625,1.074685,1.074615,1.074665,-1.0,-1.0,-1
676,2023-11-06 09:31:00+00:00,1.074665,1.074695,1.074615,1.074615,-1.0,-1.0,-1


In [10]:
# We want to back test various FX strategies using simple moving average. Theory tells us that if SMA_short < SMA_long, we
# should buy, while the converse is true. This strategy tests for the theoretical SMA crossover using '1 min' as freq

results = []

for i in range(2, 30):
    for j in range(i+1, 60):
        fx = df[['date', 'close']].copy()
        fx['sma_s'] = fx.close.rolling(i).mean()
        fx['sma_l'] = fx.close.rolling(j).mean()
        fx["position"] = np.where(fx["sma_s"] > fx["sma_l"], 1, -1)
        fx['returns'] = np.log(fx.close.div(fx.close.shift(1)))
        fx["strategy"] = fx["position"].shift(1) * fx["returns"]
        fx.dropna(inplace=True)
        fx["creturns"] = fx["returns"].cumsum().apply(np.exp)
        fx["cstrategy"] = fx["strategy"].cumsum().apply(np.exp)
        perf = fx["cstrategy"].iloc[-1] #absolute performance
        outperf = perf - fx["creturns"].iloc[-1]
        results.append(((i,j), perf, outperf))
results_df = pd.DataFrame(sorted(results, key=lambda x: x[1], reverse=True), columns = ['trading pair', 'returns', 'outperformance'])
results_df.head(10)

,trading pair,returns,outperformance
0,"(4, 6)",1.003617,0.001902
1,"(4, 5)",1.003361,0.001640
2,"(2, 8)",1.003066,0.001360
3,"(3, 5)",1.003026,0.001306
4,"(2, 5)",1.002766,0.001046
5,"(3, 8)",1.002552,0.000845
6,"(5, 8)",1.002290,0.000584
7,"(2, 7)",1.002213,0.000483
8,"(5, 9)",1.001870,0.000182
9,"(4, 7)",1.001810,0.000080


In [11]:
# As mentioned above, theory tells us that if SMA_short < SMA_long, we should buy, while the converse is true. 
# This strategy adopts a contrarian approach (inverting trade strategy) for the theoretical SMA crossover using '1 min' as freq

fx = df[['date', 'close']].copy()

results = []

for i in range(2, 30):
    for j in range(i+1, 60):
        fx = df[['date', 'close']].copy()
        fx['sma_s'] = fx.close.rolling(i).mean()
        fx['sma_l'] = fx.close.rolling(j).mean()
        fx["position"] = np.where(fx["sma_s"] < fx["sma_l"], 1, -1) # Note that this is the part where we change the strategy
        fx['returns'] = np.log(fx.close.div(fx.close.shift(1)))
        fx["strategy"] = fx["position"].shift(1) * fx["returns"]
        fx.dropna(inplace=True)
        fx["creturns"] = fx["returns"].cumsum().apply(np.exp)
        fx["cstrategy"] = fx["strategy"].cumsum().apply(np.exp)
        perf = fx["cstrategy"].iloc[-1] #absolute performance
        outperf = perf - fx["creturns"].iloc[-1]
        results.append(((i,j), perf, outperf))
results_df = pd.DataFrame(sorted(results, key=lambda x: x[1], reverse=True), columns = ['trading pair', 'returns', 'outperformance'])
results_df.head(10)

,trading pair,returns,outperformance
0,"(6, 16)",1.006602,0.004966
1,"(14, 24)",1.006588,0.005003
2,"(12, 23)",1.006550,0.004910
3,"(12, 21)",1.006527,0.004938
4,"(10, 13)",1.006501,0.004851
5,"(12, 27)",1.006418,0.004861
6,"(5, 20)",1.006415,0.004825
7,"(13, 21)",1.006340,0.004751
8,"(17, 21)",1.006199,0.004610
9,"(12, 24)",1.006185,0.004600


In [12]:
# Evaluating best theoretical SMA crossover strategy

sma_s = 2
sma_l = 5

fx = df[['date', 'close']].copy()
fx['sma_s'] = fx.close.rolling(sma_s).mean()
fx['sma_l'] = fx.close.rolling(sma_l).mean()
fx["position"] = np.where(fx["sma_s"] > fx["sma_l"], 1, -1)
fx['returns'] = np.log(fx.close.div(fx.close.shift(1)))
fx["strategy"] = fx["position"].shift(1) * fx["returns"]
fx.dropna(inplace=True)
fx["creturns"] = fx["returns"].cumsum().apply(np.exp)
fx["cstrategy"] = fx["strategy"].cumsum().apply(np.exp)
perf = fx["cstrategy"].iloc[-1] #absolute performance
outperf = perf - fx["creturns"].iloc[-1]
fx

,date,close,sma_s,sma_l,position,returns,strategy,creturns,cstrategy
4,2023-11-05 22:19:00+00:00,1.072475,1.072472,1.072475,-1,0.000005,-0.000005,1.000005,0.999995
5,2023-11-05 22:20:00+00:00,1.072460,1.072468,1.072472,-1,-0.000014,0.000014,0.999991,1.000009
6,2023-11-05 22:21:00+00:00,1.072485,1.072472,1.072471,1,0.000023,-0.000023,1.000014,0.999986
7,2023-11-05 22:22:00+00:00,1.072505,1.072495,1.072479,1,0.000019,0.000019,1.000033,1.000005
8,2023-11-05 22:23:00+00:00,1.072505,1.072505,1.072486,1,0.000000,0.000000,1.000033,1.000005
...,...,...,...,...,...,...,...,...,...
673,2023-11-06 09:28:00+00:00,1.074605,1.074698,1.074735,-1,-0.000172,-0.000172,1.001991,1.002496
674,2023-11-06 09:29:00+00:00,1.074625,1.074615,1.074704,-1,0.000019,-0.000019,1.002009,1.002477
675,2023-11-06 09:30:00+00:00,1.074665,1.074645,1.074703,-1,0.000037,-0.000037,1.002047,1.002440
676,2023-11-06 09:31:00+00:00,1.074615,1.074640,1.074660,-1,-0.000047,0.000047,1.002000,1.002486


In [13]:
# Evaluating best theoretical SMA crossover contrarian strategy

sma_s = 14
sma_l = 24

fx = df[['date', 'close']].copy()
fx['sma_s'] = fx.close.rolling(sma_s).mean()
fx['sma_l'] = fx.close.rolling(sma_l).mean()
fx["position"] = np.where(fx["sma_s"] < fx["sma_l"], 1, -1)
fx['returns'] = np.log(fx.close.div(fx.close.shift(1)))
fx["strategy"] = fx["position"].shift(1) * fx["returns"]
fx.dropna(inplace=True)
fx["creturns"] = fx["returns"].cumsum().apply(np.exp)
fx["cstrategy"] = fx["strategy"].cumsum().apply(np.exp)
perf = fx["cstrategy"].iloc[-1] #absolute performance
outperf = perf - fx["creturns"].iloc[-1]
fx

,date,close,sma_s,sma_l,position,returns,strategy,creturns,cstrategy
23,2023-11-05 22:38:00+00:00,1.072575,1.072570,1.072533,-1,-0.000037,0.000037,0.999963,1.000037
24,2023-11-05 22:39:00+00:00,1.072575,1.072576,1.072537,-1,0.000000,-0.000000,0.999963,1.000037
25,2023-11-05 22:40:00+00:00,1.072645,1.072583,1.072544,-1,0.000065,-0.000065,1.000028,0.999972
26,2023-11-05 22:41:00+00:00,1.072635,1.072589,1.072551,-1,-0.000009,0.000009,1.000019,0.999981
27,2023-11-05 22:42:00+00:00,1.072690,1.072599,1.072560,-1,0.000051,-0.000051,1.000070,0.999930
...,...,...,...,...,...,...,...,...,...
673,2023-11-06 09:28:00+00:00,1.074605,1.074975,1.075048,1,-0.000172,-0.000172,1.001855,1.006860
674,2023-11-06 09:29:00+00:00,1.074625,1.074921,1.075030,1,0.000019,0.000019,1.001874,1.006879
675,2023-11-06 09:30:00+00:00,1.074665,1.074871,1.075013,1,0.000037,0.000037,1.001911,1.006916
676,2023-11-06 09:31:00+00:00,1.074615,1.074801,1.074995,1,-0.000047,-0.000047,1.001865,1.006869
